# Lógica Computacional 24/25

**Grupo 09**

- João Afonso Almeida Sousa (A102462)
- Rafael Cunha Costa (A102526)

Um  programa imperativo pode ser descrito por um modelo do tipo Control Flow Automaton (CFA) como a seguir se indica.

<img src="J0oAAOLw.png" width="500" />

## Pretende-se:

Construir um SFOTS, usando BitVec’s de tamanho $n$ , que descreva o comportamento deste autómato; para isso identifique e codifique em Z3  ou pySMT, as variáveis do modelo, o estado inicial , a relação de transição e o estado de erro.


Usando $k$-indução verifique nesse SFOTS se a propriedade $\;(x*y + z = a*b)\;$ é um invariante do seu comportamento.

Usando $k$-indução no FOTS acima e adicionando ao estado inicial  a condição  $\,(a < 2^{n/2})\land(b < 2^{n/2})\,$, verifique a segurança do programa; nomeadamente  prove que, com tal estado inicial, o estado de erro nunca é acessível.  

## Inicialização



In [130]:
from z3 import *

## Definição das variáveis do modelo

A primeira função define as variáveis de estado para o nosso modelo, usando BitVec e Int para representar as variáveis do autômato e o ponteiro de controlo (pc). Além disso, define as variáveis x, y, z, que são as variáveis do programa.

In [131]:
def declare(i, n):
    state = {}
    state['pc'] = Int('pc'+str(i))
    state['x'] = BitVec('x'+str(i), n)
    state['y'] = BitVec('y'+str(i), n)
    state['z'] = BitVec('z'+str(i), n)


    return state

## Definição do Estado Inicial

A função *init* define o estado inicial do programa no autômato. Ela configura as variáveis de estado da seguinte forma:

- **pc** (ponteiro de controlo) é definido como `0`, indicando o ponto de partida.
- **x**, **y**, e **z** são as variáveis do programa:
  - **x** é inicializado com o valor de `a`.
  - **y** é inicializado com o valor de `b`.
  - **z** é inicializado como `0`, pois ele acumulará o resultado da multiplicação.

A função também garante que os valores iniciais de `a` e `b` sejam menores que $2^n / 2^2n$, para evitar problemas de overflow durante a execução do programa.


In [132]:
def init(state, a, b):
    return And(state['pc']== 0, state['x'] == a, state['y'] == b, state['z'] == 0)


## Definição da relação de transição

A função *trans* define as transições possíveis entre estados no modelo do autômato. A transição depende do valor do ponteiro de controlo `(pc)`, que determina qual operação será realizada com as variáveis `x`, `y` e `z`.

In [133]:
def trans(curr, prox, n):
    same_values = And(
        prox['x'] == curr['x'],
        prox['y'] == curr['y'],
        prox['z'] == curr['z']    
    )
    
    t0 = And(
        curr['pc'] == 0,
        prox['pc'] == 1,
        same_values
    )
    
    # y = 0
    t1 = And(
        curr['y'] == 0,
        curr['pc'] == 1,
        prox['pc'] == 5,
        same_values
    )
    
    # y != 0 ^ odd(y)
    t2 = And(
        curr['y'] != 0,
        URem(curr['y'], 2) == 1,
        curr['pc'] == 1,
        prox['pc'] == 2,
        same_values
    )
    
    #Q2 -> Q1
    t5 = And(
        curr['pc'] == 2,
        prox['pc'] == 1,
        prox['x'] == curr['x'],
        prox['y'] == curr['y']-1,
        prox['z'] == curr['z'] + curr['x']
    )
    
    # y != 0 ^ even(y)
    t3 = And(
        curr['y'] != 0,
        URem(curr['y'], 2) == 0,
        curr['pc'] == 1,
        prox['pc'] == 3,
        same_values
    )


    # transição em que o solver decide se vai para o estado de overflow ou se continua
    decider = And(
        prox['x'] == curr['x'] << BitVecVal(1, n),
        prox['y'] == curr['y'] >> BitVecVal(1, n),
        prox['z'] == curr['z'],
        
        curr['pc'] == 3,
        
        Or(
            And(UGT(curr['x'], BitVecVal(2**(n-1), n)), prox['pc'] == 1),  # curr['x'] < 2^(n-1)  - não há overflow
            And(ULT(curr['x']  >> BitVecVal(1, n), curr['x']), prox['pc'] == 4),  # curr['x'] >= 2^(n-1) - há overflow
        )
    )
    
    
    # caso de paragem no overflow e no estado final
    stop_case = And(
        prox['pc'] == curr['pc'],
        same_values,
        
        Or(
            And(curr['pc'] == 4, prox['pc'] == 4),
            And(curr['pc'] == 5, prox['pc'] == 5)
        )
    )
    
    
    return Or(t0, t1, t2, t3, t5, stop_case, decider)

## Geração de um traço (sequência de estados)

A função *gera_traco* gera um traço (sequência de estados) de k passos. A função também adiciona o estado inicial e as transições entre os estados para verificar o comportamento do autômato ao longo dos passos.

In [134]:
def gera_traco(declare,init,trans,k, n, a, b):

    s = Solver()
    
    
    trace = [declare(i, n) for i in range(k)]

    # adicionar o estado inicial
    s.add(init(trace[0],a,b))
    
    # adicionar as transições
    for i in range(k - 1):
        s.add(trans(trace[i], trace[i+1], n))
    
    
    check = s.check()
    if check == sat:
        m = s.model()
        #print(m)
        #print(trace[i])
        for i in range(k):
            print("Passo ", i)
            for v in trace[i]:
                
                print(v, "=", m[trace[i][v]])
            print("----------------")
    else:
        print(check)
                
gera_traco(declare,init,trans,20, 8, 150, 2)


Passo  0
pc = 0
x = 150
y = 2
z = 0
----------------
Passo  1
pc = 1
x = 150
y = 2
z = 0
----------------
Passo  2
pc = 3
x = 150
y = 2
z = 0
----------------
Passo  3
pc = 1
x = 44
y = 1
z = 0
----------------
Passo  4
pc = 2
x = 44
y = 1
z = 0
----------------
Passo  5
pc = 1
x = 44
y = 0
z = 44
----------------
Passo  6
pc = 5
x = 44
y = 0
z = 44
----------------
Passo  7
pc = 5
x = 44
y = 0
z = 44
----------------
Passo  8
pc = 5
x = 44
y = 0
z = 44
----------------
Passo  9
pc = 5
x = 44
y = 0
z = 44
----------------
Passo  10
pc = 5
x = 44
y = 0
z = 44
----------------
Passo  11
pc = 5
x = 44
y = 0
z = 44
----------------
Passo  12
pc = 5
x = 44
y = 0
z = 44
----------------
Passo  13
pc = 5
x = 44
y = 0
z = 44
----------------
Passo  14
pc = 5
x = 44
y = 0
z = 44
----------------
Passo  15
pc = 5
x = 44
y = 0
z = 44
----------------
Passo  16
pc = 5
x = 44
y = 0
z = 44
----------------
Passo  17
pc = 5
x = 44
y = 0
z = 44
----------------
Passo  18
pc = 5
x = 44
y = 0
z = 44
---

## Definição do Invariante

O invariante nonnegative que queremos verificar é a propriedade de que $x * y + z == a * b$, ou seja, queremos garantir que essa relação se mantenha durante a execução do programa.

In [135]:
# Definir o invariante nonnegative como (x * y + z == a * b)
def nonnegative(state, a, b):
    return state['x'] * state['y'] + state['z'] == a * b

## Função de k-indução

A função *kinduction_always* realiza a verificação de *k-indução*, onde verificamos que a propriedade *inv* (invariante) se mantém ao longo de *k* estados. Primeiro, verificamos o caso base e, em seguida, usamos indução para garantir que a propriedade seja válida para o próximo estado.

In [136]:
def kinduction_always(declare, init, trans, inv, k, n, a_val, b_val):
    solver = Solver()
    
    # Passo Base: Verificar que a propriedade se mantém para os primeiros k estados
    trace = [declare(i, n) for i in range(k)]
    a = BitVecVal(a_val, n)
    b = BitVecVal(b_val, n)
    
    solver.add(init(trace[0], a, b))
    
    for i in range(k - 1):
        solver.add(trans(trace[i], trace[i + 1], n))
        
    for i in range(k):
        solver.push()
        solver.add(Not(inv(trace[i], a, b)))  # Verificar se o inv é falso em algum estado
        if solver.check() == sat:
            print("> O invariante não se verifica nos k estados iniciais.")
            for j in range(k):
                print(f"> Estado {j}: x = {solver.model()[trace[j]['x']]}, y = {solver.model()[trace[j]['y']]}, z = {solver.model()[trace[j]['z']]}, pc = {solver.model()[trace[j]['pc']]}")
            return
        solver.pop()

    # Passo Indutivo: Se inv se verifica para os k estados, então deve valer no próximo (k+1)-ésimo estado
    extended_trace = [declare(i, n) for i in range(k + 1)]
    
    solver.add(init(extended_trace[0], a, b))

    for i in range(k):
        solver.add(inv(extended_trace[i], a, b))
        solver.add(trans(extended_trace[i], extended_trace[i + 1], n))
    
    solver.add(Not(inv(extended_trace[-1], a, b)))
    
    if solver.check() == sat:
        print("> O passo indutivo não se verifica.")
        for i in range(k + 1):
            print(f"> Estado {i}: x = {solver.model()[extended_trace[i]['x']]}, y = {solver.model()[extended_trace[i]['y']]}, z = {solver.model()[extended_trace[i]['z']]}, pc = {solver.model()[extended_trace[i]['pc']]}")
        return
    
    print(f"> A propriedade verifica-se por k-indução (k={k}).")



## Verificar a propriedade
Chamamos a função *kinduction_always* para verificar a propriedade com *k-indução*. 
Neste caso, estamos a verificar a propriedade para k=10 passos e n=8 bits, com a = 7 e b = 4.

In [137]:
# Chamada para verificar a propriedade com k-indução
kinduction_always(declare, init, trans, nonnegative, 10, 8, 7, 4)

> A propriedade verifica-se por k-indução (k=10).


In [138]:
kinduction_always(declare, init, trans, nonnegative, 6, 8, 10, 3)

> A propriedade verifica-se por k-indução (k=6).


In [139]:
kinduction_always(declare, init, trans, nonnegative, 13, 8, 5, 8)

> A propriedade verifica-se por k-indução (k=13).
